<a href="https://colab.research.google.com/github/kimhwijin/HandsOnMachineLearing/blob/main/DimenstionalityReduction_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
import pandas as pd
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
df = pd.read_csv(url, names=['sepal length','sepal width','petal length','petal width','target'])
df.head()

,sepal length,sepal width,petal length,petal width,target
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [71]:
from sklearn.preprocessing import StandardScaler  # 표준화 패키지 라이브러리 
from sklearn.model_selection import train_test_split
import tensorflow as tf
X = df.drop(['target'], axis=1).values # 독립변인들의 value값만 추출
y = df['target'].values # 종속변인 추출

X = StandardScaler().fit_transform(X) # x객체에 x를 표준화한 데이터를 저장
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

features = ['sepal length', 'sepal width', 'petal length', 'petal width']
pd.DataFrame(X, columns=features).head()

,sepal length,sepal width,petal length,petal width
0,-0.900681,1.032057,-1.341272,-1.312977
1,-1.143017,-0.124958,-1.341272,-1.312977
2,-1.385353,0.337848,-1.398138,-1.312977
3,-1.506521,0.106445,-1.284407,-1.312977
4,-1.021849,1.263460,-1.341272,-1.312977


In [72]:
from sklearn.decomposition import PCA
import numpy as np

pca = PCA(n_components=2) # 주성분을 몇개로 할지 결정
X2D = pca.fit_transform(X)
print(pca.components_.T)
print(pca.explained_variance_ratio_) #데이터셋 분산의 72%가 첫번째축에, 23%가 두번째축에 놓여져있다.

[[ 0.52237162  0.37231836]
 [-0.26335492  0.92555649]
 [ 0.58125401  0.02109478]
 [ 0.56561105  0.06541577]]
[0.72770452 0.23030523]


In [73]:
#적절한 차원의 수 선택하기
pca = PCA()
pca.fit(X)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) + 1
print(d)

#차원의 수 지정보다, 분산 경계를 지정하면 됨.
pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(X_train)
print(pca.components_)

2
[[ 0.54169728 -0.24574475  0.57890827  0.55770852]
 [ 0.37012964  0.92807266  0.00905089  0.04004085]]


In [74]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train.astype(np.float32);X_test.astype(np.float32)
X_train = X_train / 255;X_test = X_test / 255
X_train = X_train.reshape([60000, 784])
X_test = X_test.reshape([10000, 784])

In [75]:
pca = PCA()
pca.fit(X_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) + 1
print(d)

154


In [77]:
pca = PCA(n_components=d)
X_reduced = pca.fit_transform(X_train)
X_recoverd = pca.inverse_transform(X_reduced)


In [79]:
#랜덤 PCA
rnd_pca = PCA(n_components=154, svd_solver='randomized')
X_reduced = rnd_pca.fit_transform(X_train)

In [80]:
#incremental PCA
from sklearn.decomposition import IncrementalPCA

n_batches = 100
inc_pca = IncrementalPCA(n_components=154)
for X_batch in np.array.split(X_train, n_batches):
    inc_pca.partial_fit(X_batch)

X_reduced = inc_pca.transform(X_train)


In [98]:
#파이썬 클래스 memmap 을 이용해서 자연스럽게 사용하기p
X_train = X_train.astype(np.float32)

np.save('X_train',X_train)
X_mm = np.memmap('X_train.npy', dtype='float32', mode='readonly', shape=(60000, 784))
batch_size = 60000 // n_batches
inc_pca = IncrementalPCA(n_components=154, batch_size=batch_size)
inc_pca.fit(X_mm)

/usr/local/lib/python3.7/dist-packages/numpy/lib/nanfunctions.py:1544: RuntimeWarning: overflow encountered in multiply
  sqr = np.multiply(arr, arr, out=arr)
/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_incremental_pca.py:296: RuntimeWarning: overflow encountered in square
  explained_variance = S ** 2 / (n_total_samples - 1)
/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_incremental_pca.py:297: RuntimeWarning: overflow encountered in square
  explained_variance_ratio = S ** 2 / np.sum(col_var * n_total_samples)
/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_incremental_pca.py:297: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio = S ** 2 / np.sum(col_var * n_total_samples)


IncrementalPCA(batch_size=600, copy=True, n_components=154, whiten=False)